## Importing Libraries

In [1]:
!pip install -q -U albumentations
!pip install -q -U datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10

In [2]:
import numpy as np
import pandas as pd
import torch 
from pathlib import Path
from PIL import Image

import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import MNIST

import os
import csv
from torchvision import transforms
import albumentations
from typing import Tuple, List, Optional

In [3]:
!git clone https://github.com/VAaRrUNn/HumanAI_Gsoc.git

Cloning into 'HumanAI_Gsoc'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 35 (delta 3), reused 29 (delta 1), pack-reused 0
Unpacking objects: 100% (35/35), 7.77 KiB | 995.00 KiB/s, done.


In [4]:
cd HumanAI_Gsoc/Data

/kaggle/working/HumanAI_Gsoc/Data


In [5]:
ls

dataset.csv  images/  text.txt


In [6]:
class temp(Dataset):
    def __init__(self):
        super().__init__()
        
    def __len__(self):
        return 10;
        
    def __getitem__(self, idx):
        return 10, 20
    
td = temp()

def wow(x):
    print("In wow")
    print(x)
    print("Out of wow")
tdl = DataLoader(dataset = td,
                batch_size = 2,
                collate_fn = wow)

next(iter(tdl))

In wow
[(10, 20), (10, 20)]
Out of wow


In [9]:
# Load model directly
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

sen = ["what are you doing", "My name is varun "]
t1 = tokenizer("what are you doing")
t2 = tokenizer("My name is varun ")
tt = tokenizer(sen)
# data_collator([t1, t2])
data_collator(tt)

{'input_ids': tensor([[   96,  6076, 24554, 17257,   728,   640, 50256],
        [12330,   309,  2002,  1672,   958,   290,   230]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1]])}

## Making dataset
Dataset structure:
#### 1. images/
- this contains all the images in jpeg format
#### 2. dataset.csv
- It contains 2 cols
- This contains the images name which are present in the images/ folder and their corrosponding text

In [25]:
class OCR_temp(Dataset):
    def __init__(self,
                images,
                labels,
                transforms = None,
                tokenizer = None):
        super().__init__()
        self.images = images
        self.labels = labels
        
        self.transforms = transforms
        self.tokenizer = tokenizer
        
    def __len__(self,) -> int:
        return len(self.images)
    
    def __getitem__(self,
                   idx) -> Tuple:
        img, lab = Image.open(self.images[idx]), self.labels[idx]
        if self.transforms:
            img = self.transforms(img)
        
        # The padding will be done in coll_fn in dataloader
        lab = self.tokenizer(lab)
        
        return (img, lab)
        
        

class OCRDataModule(pl.LightningDataModule):
    
    def __init__(self,
                data_dir: str,
                csv_file,
                transforms,
                batch_size = 5,
                tokenizer = None,
                data_collator = None):
        super().__init__()
        self.batch_size = batch_size
        self.data_dir = Path(data_dir)
        self.csv_file = csv_file 
        
        self.transforms = transforms
        self.tokenizer = tokenizer
        self.data_collator = data_collator
        
        
    def prepare_data(self):
        """
        Some Preprocessing
        """
        images_name, self.labels = [], []
        with open("dataset.csv", 'r') as f:
            content = csv.reader(f)
            
            # Skipping 1st row
            next(content)
            
            for row in content:
                images_name.append(row[0])
                self.labels.append(row[1])
        
        
        
        # Getting absolute path for images
        self.images = [
            self.data_dir / Path("images") / x for x in images_name
        ]
        
    
    def setup(self,
             stage: str = "train"):
        if stage == "fit" or "validate":
            self.data = OCR_temp(images = self.images,
                            labels = self.labels,
                            transforms = self.transforms,
                            tokenizer = self.tokenizer)
        
        if stage == "test":
            """
            Currently not implemented, because the (current) dataset size is very small
            """
            pass
    
    def __collate_fn(self,
                    samples) -> Tuple:
        images, labels = [], []
        for (img, lab) in samples:
            images.append(img)
            labels.append(lab)
        labels = self.data_collator(labels)
        return (images, labels)
            
        
            
    def train_dataloader(self):
        return DataLoader(dataset = self.data,
                         batch_size = self.batch_size,
                         shuffle = True,
                         collate_fn = self.__collate_fn)
        
    def val_dataloader(self):
        pass
    
    def test_dataloader(self):
        pass
    

In [26]:
# Load model directly
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

# some testing
# t1 = tokenizer("what are you doing")
# t2 = tokenizer("My name is varun ")
# data_collator([t1, t2])

train_transform = transforms.Compose([
    transforms.ToTensor(),
])

tdataset = OCRDataModule(data_dir = ".", # currently it's . because I am already in the data dir, but will not be the case in general
             csv_file = "dataset.csv",
             transforms = train_transform,
             tokenizer = tokenizer,
             data_collator = data_collator)


tdataset.prepare_data()
tdataset.setup()

next(iter(tdataset.train_dataloader()))

([tensor([[[0.0157, 0.0157, 0.0157,  ..., 0.0118, 0.0314, 0.0314],
           [0.0118, 0.0118, 0.0118,  ..., 0.0118, 0.0314, 0.0314],
           [0.0471, 0.0471, 0.0471,  ..., 0.0118, 0.0353, 0.0353],
           ...,
           [0.0314, 0.0235, 0.0118,  ..., 0.0196, 0.0314, 0.0353],
           [0.0353, 0.0353, 0.0235,  ..., 0.0039, 0.0157, 0.0157],
           [0.0353, 0.0353, 0.0235,  ..., 0.0039, 0.0157, 0.0157]],
  
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0275, 0.0275, 0.0275,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0157, 0.0078, 0.0000,  ..., 0.0157, 0.0157, 0.0157],
           [0.0157, 0.0157, 0.0078,  ..., 0.0000, 0.0000, 0.0039],
           [0.0157, 0.0157, 0.0078,  ..., 0.0000, 0.0000, 0.0039]],
  
          [[0.0118, 0.0118, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
           [0.0078, 0.0078, 0.0078,  ..., 0.0000, 0.0000, 0.0000],
           [0.0510, 0.